In [4]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
#import tim
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine
#https://chmodi.tistory.com/114

In [2]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


In [6]:
#DB 에서 데이터 가져오기

curs.execute("select * from mimiciv_derived.vitalsign;")
columns_name = [desc[0] for desc in curs.description]


In [7]:
vitalsign = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [8]:
vitalsign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9745500 entries, 0 to 9745499
Data columns (total 15 columns):
 #   Column            Dtype         
---  ------            -----         
 0   subject_id        int64         
 1   stay_id           int64         
 2   charttime         datetime64[ns]
 3   heart_rate        float64       
 4   sbp               float64       
 5   dbp               float64       
 6   mbp               float64       
 7   sbp_ni            float64       
 8   dbp_ni            float64       
 9   mbp_ni            float64       
 10  resp_rate         float64       
 11  temperature       object        
 12  temperature_site  object        
 13  spo2              float64       
 14  glucose           float64       
dtypes: datetime64[ns](1), float64(10), int64(2), object(2)
memory usage: 1.1+ GB


### 환자정보와 join

In [9]:
subjectlist1 = pd.read_csv('./data/sub1list.csv', parse_dates=['admittime','dischtime'])
subjectlist1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6725 entries, 0 to 6724
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  6725 non-null   int64         
 1   subject_id  6725 non-null   int64         
 2   hadm_id     6725 non-null   int64         
 3   stay_id     6724 non-null   float64       
 4   admittime   6725 non-null   datetime64[ns]
 5   dischtime   6725 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3)
memory usage: 315.4 KB


In [10]:
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')

In [11]:
sub1_mbp = sub1list >> left_join(vitalsign, by=['subject_id','stay_id']) >> select('subject_id', 'stay_id', 'charttime', 'mbp')

### mbp null 값 제거, 중복값 확인
- 주의: vital 데이터의 경우 완전 중복값이 아닌 근접값도 고려할 필요 있음

In [12]:
# null 값 확인
sub1_mbp_notna = sub1_mbp[sub1_mbp.mbp.notna()]
sub1_mbp_notna

,subject_id,stay_id,charttime,mbp
3,10001884,37510196.0,2131-01-11 04:25:00,46.0
5,10001884,37510196.0,2131-01-11 05:01:00,70.0
7,10001884,37510196.0,2131-01-11 06:01:00,75.0
11,10001884,37510196.0,2131-01-11 06:47:00,84.0
13,10001884,37510196.0,2131-01-11 07:01:00,73.0
...,...,...,...,...
2090334,19999840,38978960.0,2164-09-17 12:00:00,65.0
2090335,19999840,38978960.0,2164-09-17 13:00:00,45.0
2090336,19999840,38978960.0,2164-09-17 13:05:00,51.0
2090337,19999840,38978960.0,2164-09-17 13:10:00,33.0


In [13]:
# null 값 제거 후 stay id 카운트 동일.

print(f'stay_id COUNT BEFORE: {sub1_mbp.stay_id.nunique()}')
print(f'stay_id COUNT AFTER: {sub1_mbp_notna.stay_id.nunique()}')

stay_id COUNT BEFORE: 6724
stay_id COUNT AFTER: 6724


In [14]:
# 중복값 확인 --> 없음
## 중복값 정의: stay_id 내에서 charttime 동일한 경우
duplicates = sub1_mbp_notna[sub1_mbp_notna.duplicated(subset=['stay_id', 'charttime'], keep=False)]
duplicates_sorted = duplicates.sort_values(by=['stay_id', 'charttime'])   # optional

duplicates_sorted


,subject_id,stay_id,charttime,mbp


In [16]:
sub1_mbp_notna.to_csv('../outputs/sub1_mbp_notna.csv', index=False, encoding='utf-8-sig')
